# Smart Care Analytics

## Packages 

In [9]:
import pdfplumber
import pandas as pd
import re
import os
import openpyxl



In [11]:
# --- CONFIGURATION ---
# Remplacez par le nom exact de votre fichier. 
# Note : Le fichier fourni est "SLP-CHF2012.pdf", mais vous avez mentionné 2021.
PDF_PATH = "fichier/SLP-CHF2012.pdf" 
OUTPUT_EXCEL = "Rapport_Donnees_Extraites.xlsx"

In [12]:


def nettoyer_donnees(df):
    """
    Fonction de nettoyage spécifique pour les tableaux de rapports :
    1. Remplace les retours à la ligne (\n) par des espaces.
    2. Supprime les colonnes et lignes entièrement vides.
    """
    # Remplacer les sauts de ligne internes (ex: "Chiffre\nd'affaires" -> "Chiffre d'affaires")
    df = df.replace(r'\n', ' ', regex=True)
    
    # Remplacer les valeurs None (vides) par des chaînes vides pour éviter les erreurs
    df = df.fillna("")
    
    # Supprimer les lignes où TOUTES les colonnes sont vides
    df = df.dropna(how='all')
    
    # Supprimer les colonnes où TOUTES les lignes sont vides
    df = df.dropna(axis=1, how='all')
    
    return df

def extraire_tableaux_pdf(pdf_path):
    tableaux_finaux = []
    
    print(f"--- Démarrage de l'extraction sur : {pdf_path} ---")
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            # Boucle sur chaque page du PDF
            for i, page in enumerate(pdf.pages):
                print(f"Traitement de la page {i + 1}...")
                
                # Extraction des tableaux sur la page actuelle
                tables = page.extract_tables()
                
                for table in tables:
                    # Convertir la liste brute en DataFrame Pandas
                    df = pd.DataFrame(table)
                    
                    # Nettoyage préliminaire
                    df = nettoyer_donnees(df)
                    
                    # Vérification : on ne garde que les tableaux qui contiennent des données
                    if not df.empty and len(df) > 1:
                        tableaux_finaux.append(df)
                        
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return []

    return tableaux_finaux

def supprimer_doublons_et_formater(liste_dfs):
    """
    Parcourt la liste des DataFrames extraits pour supprimer les doublons exacts
    et formater les en-têtes.
    """
    resultats_uniques = []
    signatures_vues = set()

    for df in liste_dfs:
        # Création d'une signature unique pour le tableau (basée sur son contenu)
        # Cela permet de détecter si le même tableau est extrait deux fois
        signature = df.to_json()
        
        if signature not in signatures_vues:
            signatures_vues.add(signature)
            
            # --- Formatage des en-têtes ---
            # Si la première ligne ressemble à un en-tête (texte non vide), on la promeut
            first_row = df.iloc[0].astype(str).tolist()
            # Heuristique simple : si la ligne 0 n'a pas trop de cellules vides
            if sum(len(x) > 0 for x in first_row) > 1:
                new_header = df.iloc[0] # Prendre la première ligne
                df = df[1:] # Prendre les données à partir de la ligne suivante
                df.columns = new_header # Assigner l'en-tête
            
            resultats_uniques.append(df)
            
    return resultats_uniques

def sauvegarder_excel(liste_dfs, output_path):
    if not liste_dfs:
        print("Aucune donnée n'a été extraite.")
        return

    print(f"--- Sauvegarde de {len(liste_dfs)} tableaux uniques dans {output_path} ---")
    
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        for index, df in enumerate(liste_dfs):
            # Nommer l'onglet (Sheet1, Sheet2, etc.)
            nom_onglet = f"Tableau_{index + 1}"
            df.to_excel(writer, sheet_name=nom_onglet, index=False)
            
    print("Terminé avec succès !")


In [13]:

# --- EXÉCUTION DU SCRIPT ---
if __name__ == "__main__":
    if os.path.exists(PDF_PATH):
        # 1. Extraction
        dfs_bruts = extraire_tableaux_pdf(PDF_PATH)
        
        # 2. Suppression des doublons et Formatage
        dfs_propres = supprimer_doublons_et_formater(dfs_bruts)
        
        # 3. Sauvegarde
        sauvegarder_excel(dfs_propres, OUTPUT_EXCEL)
    else:
        print(f"Erreur : Le fichier {PDF_PATH} est introuvable.")

--- Démarrage de l'extraction sur : fichier/SLP-CHF2012.pdf ---
Traitement de la page 1...
Traitement de la page 2...
Traitement de la page 3...
Traitement de la page 4...
--- Sauvegarde de 14 tableaux uniques dans Rapport_Donnees_Extraites.xlsx ---
Terminé avec succès !
